### GW waveform : parameter estimation
$$ S(t) = h(t) + n(t) \quad h(t,A,f_0,\phi_0) = A sin(2\pi f_0t + \phi_0) $$
- $n(t)$ is gaussian
- $S_n(f) = 10^{-42} Hz^{-1}$
- $ T = 10^8 s$
- $ A = 10^{-22} s$
- $ f = 10^{-2} Hz$
- $\phi_0 = \pi$

To what precision you can measure $A,f_0,\phi_0$ ?
What happens if you change $ A (10^{-23} ,10^{-21}), T (10^{7} ,10^{9}), f(10^{-3}) $ ?


In [1]:
import sympy as sp
import numpy as np
import math
from sympy import *

In [2]:
def compute_SNR(SNR,A_v, f0_v, phi0_v, T_v, Sn_v):
    SNR = SNR.subs([(A, A_v),(f0, f0_v),(phi0, phi0_v),(T,T_v), (Sn,Sn_v)])
    return SNR
    

def compute_err(F,A_v, f0_v, phi0_v, T_v, Sn_v):
        
    F_v = Matrix(3, 3, lambda i,j: F[i,j].subs([(A, A_v),(f0, f0_v),(phi0, phi0_v),(T,T_v), (Sn,Sn_v)]))
    Sigma = F_v.inv()
    cov = Sigma.evalf()
    err_abs = np.array([(cov[i,i])**(1/2) for i in range(3)])
    err_rel = err_abs/[A_v, f0_v, phi0_v]
    
    return err_abs.astype(np.float64), err_rel.astype(np.float64)

In [3]:
t, A, f0, phi0, T, Sn = sp.symbols('t A f0 phi0 T Sn')
h = A*sin(2*pi*f0*t + phi0)

A_v = 10**(-22)
f0_v = 10**(-2)
phi0_v = pi
T_v = 10**8
Sn_v = 10**(-42)

SNR = (integrate((2/Sn)*h*h,(t,0,T)))**(1/2)
par = [A, f0, phi0]
D = Matrix([diff(h,par[i]) for i in range(3)])
F = Matrix(3, 3, lambda i,j: integrate((2/Sn)*D[i]*D[j],(t,0,T)))

SNR_v = compute_SNR(SNR, A_v, f0_v, phi0_v, T_v, Sn_v)
print('S/N : %.0f' %SNR_v)

err_abs, err_rel = compute_err(F, A_v, f0_v, phi0_v, T_v, Sn_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))

S/N : 1000
Abs error :  [1.000e-25 5.513e-12 2.000e-03]
Rel error :  [1.000e-03 5.513e-10 6.366e-04]


### Change A

In [4]:
A_v1 = 10**(-23)
A_v2 = 10**(-21)

err_abs, err_rel = compute_err(F, A_v1, f0_v, phi0_v, T_v, Sn_v)
SNR_v = compute_SNR(SNR, A_v1, f0_v, phi0_v, T_v, Sn_v)

print(A, ' = %.3e' %A_v1)
print('S/N : %.0f' %SNR_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))
print('\n')

err_abs, err_rel = compute_err(F, A_v2, f0_v, phi0_v, T_v, Sn_v)

SNR_v = compute_SNR(SNR, A_v2, f0_v, phi0_v, T_v, Sn_v)
print(A, ' = %.3e' %A_v2)
print('S/N : %.0f' %SNR_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))
print('\n')

print('By increasing A: the S/N increases linearly and the relative error on the parameters decreases linearly')

A  = 1.000e-23
S/N : 100
Abs error :  [1.000e-25 5.513e-11 2.000e-02]
Rel error :  [1.000e-02 5.513e-09 6.366e-03]


A  = 1.000e-21
S/N : 10000
Abs error :  [1.000e-25 5.513e-13 2.000e-04]
Rel error :  [1.000e-04 5.513e-11 6.366e-05]


By increasing A: the S/N increases linearly and the relative error on the parameters decreases linearly


### Change T

In [5]:
T_v1 = 10**7
T_v2 = 10**9

err_abs, err_rel = compute_err(F, A_v, f0_v, phi0_v, T_v1, Sn_v)
SNR_v = compute_SNR(SNR, A_v, f0_v, phi0_v, T_v1, Sn_v)

print(T, ' = %.0e' %T_v1)
print('S/N : %.1f' %SNR_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))
print('\n')

err_abs, err_rel = compute_err(F, A_v, f0_v, phi0_v, T_v2, Sn_v)
SNR_v = compute_SNR(SNR, A_v, f0_v, phi0_v, T_v2, Sn_v)

print(T, ' = %.0e' %T_v2)
print('S/N : %.0f' %SNR_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))
print('\n')

print('By increasing T: the S/N increases and the error on the parameters decreases, both as T^1/2,  because the error on the parameters follow the scaling of the SNR')

T  = 1e+07
S/N : 316.2
Abs error :  [3.162e-25 1.743e-10 6.325e-03]
Rel error :  [3.162e-03 1.743e-08 2.013e-03]


T  = 1e+09
S/N : 3162
Abs error :  [3.162e-26 1.743e-13 6.325e-04]
Rel error :  [3.162e-04 1.743e-11 2.013e-04]


By increasing T: the S/N increases and the error on the parameters decreases, both as T^1/2,  because the error on the parameters follow the scaling of the SNR


### Change f

In [6]:
f0_v1 = 10**-3

err_abs, err_rel = compute_err(F, A_v, f0_v1, phi0_v, T_v, Sn_v)
SNR_v = compute_SNR(SNR, A_v, f0_v1, phi0_v, T_v, Sn_v)

print(f0, ' = %.0e' %f0_v1)
print('S/N : %.0f' %SNR_v)
print('Abs error : ', np.array_str(err_abs, precision=3))
print('Rel error : ', np.array_str(err_rel, precision=3))
print('\n')

print('By decreasing the frequency: the S/N and the error on the parameters do not change')

f0  = 1e-03
S/N : 1000
Abs error :  [1.000e-25 5.513e-12 2.000e-03]
Rel error :  [1.000e-03 5.513e-09 6.366e-04]


By decreasing the frequency: the S/N and the error on the parameters do not change
